In [1]:
import pandas as pd
import os
cur_dir = os.getcwd()
cur_dir

'/Users/garrettroell/Documents/VS_code_projects/machine_learning_clostridium/lib'

In [2]:
raw_data = pd.read_csv(f'{cur_dir}/data/raw_data.csv')
raw_data.set_index(['composition','trial','time'],drop=True,inplace=True)

smooth_data = pd.read_csv(f'{cur_dir}/data/smooth_data.csv')
smooth_data.set_index(['composition','trial','time'],drop=True,inplace=True)

In [3]:
display(raw_data.head())
display(smooth_data.head())

acetate  biomass  butanol  butyrate  ethanol  \
composition trial time                                                 
1           1     0.58    21.61     0.41     0.04      0.06    10.94   
                  0.65    44.31     0.39     0.05      0.08    15.89   
                  1.02    46.19     0.46     0.21      0.64     8.14   
                  1.67    46.16     0.49     1.18      3.64    10.81   
                  3.70    34.39     0.64     8.44      9.76    20.34   

                        flow rate     H2   CO    CO2  acetate_0  biomass_0  \
composition trial time                                                       
1           1     0.58         20  0.125  0.5  0.375  16.905029   0.429546   
                  0.65         20  0.125  0.5  0.375  16.905029   0.429546   
                  1.02         20  0.125  0.5  0.375  16.905029   0.429546   
                  1.67         20  0.125  0.5  0.375  16.905029   0.429546   
                  3.70         20  0.125  0.5  0.375  16.905029   0.429546   

                        butanol_0  butyrate_0  ethanol_0  
composition trial time                                    
1           1     0.58   0.029818    0.075529   15.89524  
                  0.65   0.029818    0.075529   15.89524  
                  1.02   0.029818    0.075529   15.89524  
                  1.67   0.029818    0.075529   15.89524  
                  3.70   0.029818    0.075529   15.89524

CO    CO2     H2    acetate   biomass   butanol  \
composition trial time                                                     
1           1     0.0   0.5  0.375  0.125  16.905029  0.429546  0.029818   
                  0.1   0.5  0.375  0.125  18.001195  0.421076  0.028260   
                  0.2   0.5  0.375  0.125  19.831073  0.410803  0.028316   
                  0.3   0.5  0.375  0.125  22.196249  0.403602  0.031357   
                  0.4   0.5  0.375  0.125  25.293925  0.403709  0.028700   

                        butyrate    ethanol  flow rate  acetate_0  biomass_0  \
composition trial time                                                         
1           1     0.0   0.075529  15.895240       20.0  16.905029   0.429546   
                  0.1   0.061033  15.607023       20.0  16.905029   0.429546   
                  0.2   0.051013  15.144135       20.0  16.905029   0.429546   
                  0.3   0.050367  14.522308       20.0  16.905029   0.429546   
                  0.4   0.035514  13.956015       20.0  16.905029   0.429546   

                        butanol_0  butyrate_0  ethanol_0  
composition trial time                                    
1           1     0.0    0.029818    0.075529   15.89524  
                  0.1    0.029818    0.075529   15.89524  
                  0.2    0.029818    0.075529   15.89524  
                  0.3    0.029818    0.075529   15.89524  
                  0.4    0.029818    0.075529   15.89524

In [4]:
from machine_learning.get_trained_models import get_trained_models

trained_models = get_trained_models(raw_data, smooth_data)

raw
gradient boosting
raw, gradient boosting, comp excluded from training: none
raw, gradient boosting, comp excluded from training: 1
raw, gradient boosting, comp excluded from training: 2
raw, gradient boosting, comp excluded from training: 3
raw, gradient boosting, comp excluded from training: 4
raw, gradient boosting, comp excluded from training: 5
raw, gradient boosting, comp excluded from training: 6
raw, gradient boosting, comp excluded from training: 7
random forest
raw, random forest, comp excluded from training: none
raw, random forest, comp excluded from training: 1
raw, random forest, comp excluded from training: 2
raw, random forest, comp excluded from training: 3
raw, random forest, comp excluded from training: 4
raw, random forest, comp excluded from training: 5
raw, random forest, comp excluded from training: 6
raw, random forest, comp excluded from training: 7
support vector
raw, support vector, comp excluded from training: none
raw, support vector, comp excluded from 

In [6]:
from machine_learning.get_model_predictions import get_model_predictions
measured_time_predictions = get_model_predictions(trained_models, raw_data)
smoothed_time_predictions = get_model_predictions(trained_models, smooth_data)

In [9]:
from machine_learning.get_model_metrics import get_pearson_r2, get_rmse

def evaluate_models(model_dict, measured_values, metric):
    species_set = ['acetate', 'biomass', 'butanol', 'butyrate', 'ethanol']
    test_comp_set=[1,2,3,4,5,6,7]
    index_set = ['gradient boosting, raw', 'random forest, raw', 'support vector, raw', 'neural net (relu), raw', 'neural net (tanh), raw', 'neural net (log), raw', 'lasso, raw', 'gradient boosting, smooth', 'random forest, smooth', 'support vector, smooth', 'neural net (relu), smooth', 'neural net (tanh), smooth', 'neural net (log), smooth',  'lasso, smooth']
    
    for species in species_set:
        data = {}
        for test_comp in test_comp_set:
            data[test_comp] = []
            for model_name in model_dict.keys():
                if str(test_comp) in model_name:
                    predicted_species_values = list(measured_time_predictions[model_name].loc[test_comp][species])
                    measured_species_values = list(raw_data.loc[test_comp][species])

                    r2 = get_pearson_r2(measured_species_values, predicted_species_values)
                    rmse = get_rmse (measured_species_values, predicted_species_values)
                    if metric == 'r2':
                        data[test_comp].append(r2)
                    elif metric == 'rmse':
                        data[test_comp].append(rmse)
                    else:
                        print('unknown metric')
        species_data = pd.DataFrame.from_dict(data)
        species_data[f'model for {species}'] = index_set
        species_data.set_index(f'model for {species}', inplace=True, drop=True)
        display(species_data)

In [10]:
evaluate_models(trained_models, measured_time_predictions, 'r2')

,1,2,3,4,5,6,7
model for acetate,,,,,,,
"gradient boosting, raw",0.479696,0.774541,0.681558,0.119032,0.768161,0.281012,0.161348
"random forest, raw",0.311328,0.714856,0.490612,0.264810,0.789189,0.058443,0.080763
"support vector, raw",0.088813,0.020577,0.015806,0.053271,0.518806,0.721574,0.000000
"neural net (relu), raw",0.300245,0.102328,0.020884,0.008525,0.687905,0.952670,0.738714
"neural net (tanh), raw",0.268710,0.144872,0.027881,0.004592,0.805672,0.993968,0.713337
"neural net (log), raw",0.027194,0.083176,0.019327,0.002064,0.705700,0.974096,0.732883
"lasso, raw",0.143552,0.000000,0.007425,0.013675,0.000000,0.952670,0.000000
"gradient boosting, smooth",0.432332,0.855452,0.472822,0.054432,0.867771,0.450854,0.001614
"random forest, smooth",0.296639,0.705048,0.291366,0.208784,0.853657,0.455955,0.063120


,1,2,3,4,5,6,7
model for biomass,,,,,,,
"gradient boosting, raw",0.507182,0.913775,0.875734,0.396600,0.825967,0.509456,0.607326
"random forest, raw",0.606974,0.935656,0.932195,0.508446,0.806169,0.351008,0.534473
"support vector, raw",0.775938,0.853096,0.813017,0.278951,0.002144,0.863781,0.000000
"neural net (relu), raw",0.150998,0.160851,0.340426,0.221254,0.136892,0.553881,0.477259
"neural net (tanh), raw",0.720291,0.002823,0.062762,0.250449,0.054225,0.489728,0.504166
"neural net (log), raw",0.177977,0.003172,0.070492,0.156818,0.004045,0.564974,0.474564
"lasso, raw",0.876536,0.848818,0.690694,0.235175,0.000367,0.583200,0.477259
"gradient boosting, smooth",0.515501,0.743299,0.860293,0.339705,0.482775,0.237333,0.544273
"random forest, smooth",0.603807,0.716857,0.901871,0.372545,0.637720,0.159816,0.507525


,1,2,3,4,5,6,7
model for butanol,,,,,,,
"gradient boosting, raw",0.902343,0.702709,0.932380,0.382685,0.863012,0.140020,0.919714
"random forest, raw",0.882233,0.780788,0.934610,0.405887,0.960012,0.574470,0.982512
"support vector, raw",0.764628,0.752201,0.785126,0.604708,0.978151,0.910118,0.000000
"neural net (relu), raw",0.848555,0.814081,0.825971,0.619827,0.985715,0.854564,0.963690
"neural net (tanh), raw",0.730961,0.733249,0.790763,0.627290,0.971913,0.846356,0.965998
"neural net (log), raw",0.860026,0.667667,0.480438,0.698637,0.953358,0.878128,0.964351
"lasso, raw",0.908376,0.747660,0.850131,0.583523,0.961781,0.892565,0.965118
"gradient boosting, smooth",0.871167,0.868521,0.945363,0.521730,0.986167,0.434291,0.924258
"random forest, smooth",0.785870,0.890390,0.930185,0.406232,0.956666,0.551795,0.963249


,1,2,3,4,5,6,7
model for butyrate,,,,,,,
"gradient boosting, raw",0.564342,0.581798,0.171416,0.481905,0.523673,0.893625,0.454979
"random forest, raw",0.561185,0.576632,0.261753,0.402019,0.712589,0.915218,0.315170
"support vector, raw",0.109636,0.916591,0.790516,0.125647,0.613168,0.812240,0.000000
"neural net (relu), raw",0.098540,0.869436,0.828326,0.215221,0.565218,0.750305,0.102008
"neural net (tanh), raw",0.052271,0.824610,0.758870,0.348775,0.734048,0.891513,0.186643
"neural net (log), raw",0.374596,0.860392,0.498826,0.312272,0.569056,0.789378,0.099228
"lasso, raw",0.214721,0.678106,0.769610,0.151741,0.551598,0.775289,0.102008
"gradient boosting, smooth",0.660975,0.708595,0.319341,0.379203,0.928575,0.973466,0.614057
"random forest, smooth",0.465577,0.741048,0.249250,0.376308,0.909567,0.973689,0.477407


,1,2,3,4,5,6,7
model for ethanol,,,,,,,
"gradient boosting, raw",0.713716,0.930984,0.620015,0.404792,0.322441,0.239097,0.976161
"random forest, raw",0.763256,0.933057,0.805217,0.419370,0.658069,0.352449,0.963530
"support vector, raw",0.536626,0.785465,0.855864,0.797489,0.466480,0.479945,0.000000
"neural net (relu), raw",0.242630,0.849839,0.770208,0.834730,0.385558,0.497427,0.965253
"neural net (tanh), raw",0.633456,0.725828,0.533089,0.378906,0.209940,0.295031,0.968374
"neural net (log), raw",0.200918,0.721059,0.672054,0.643507,0.308890,0.407402,0.957625
"lasso, raw",0.376091,0.847976,0.771306,0.816436,0.359281,0.467208,0.964458
"gradient boosting, smooth",0.718921,0.687334,0.526135,0.467016,0.035147,0.298259,0.975489
"random forest, smooth",0.798724,0.954555,0.671546,0.306634,0.756764,0.456851,0.934999


In [11]:
evaluate_models(trained_models, measured_time_predictions, 'rmse')

,1,2,3,4,5,6,7
model for acetate,,,,,,,
"gradient boosting, raw",14.206500,12.294292,23.227449,22.244967,11.259894,14.621674,13.032080
"random forest, raw",17.374724,16.513877,25.901137,19.065095,10.814300,16.270370,18.141178
"support vector, raw",16.248004,25.678153,30.582719,22.354358,12.479213,13.548857,16.064278
"neural net (relu), raw",23.590153,22.846625,28.426841,15.676416,13.109106,20.863716,160.373120
"neural net (tanh), raw",25.148361,42.999157,47.593668,15.117486,22.233938,26.307272,39.613352
"neural net (log), raw",30.587304,48.672887,52.903654,20.321882,27.969341,31.802070,45.908106
"lasso, raw",21.344218,26.774492,27.253398,19.642723,11.279112,15.450178,24.260209
"gradient boosting, smooth",14.471734,16.900248,22.247911,19.764969,5.755929,15.725870,16.541422
"random forest, smooth",17.646340,20.497342,25.751775,12.447244,6.088022,20.638634,22.323339


,1,2,3,4,5,6,7
model for biomass,,,,,,,
"gradient boosting, raw",0.080118,0.098114,0.077464,0.084736,0.018747,0.057327,0.080666
"random forest, raw",0.076158,0.096946,0.088667,0.068608,0.033179,0.054978,0.085796
"support vector, raw",0.121140,0.118966,0.091255,0.079648,0.074439,0.099292,0.078410
"neural net (relu), raw",0.423808,0.099257,0.850625,0.399534,0.142003,0.097168,9.949401
"neural net (tanh), raw",0.185224,0.105372,0.767529,0.514394,0.461831,0.095439,1.715812
"neural net (log), raw",0.149575,0.142298,0.085565,0.071363,0.089569,0.094625,0.057556
"lasso, raw",0.057815,0.093394,0.098576,0.102697,0.098595,0.089008,0.059659
"gradient boosting, smooth",0.078064,0.096622,0.091720,0.076306,0.019656,0.053511,0.081616
"random forest, smooth",0.077964,0.099508,0.116275,0.078503,0.017592,0.048530,0.100725


,1,2,3,4,5,6,7
model for butanol,,,,,,,
"gradient boosting, raw",4.059043,3.013321,5.171456,4.226781,3.897974,3.324935,2.270320
"random forest, raw",4.520986,2.632481,4.377957,4.084873,3.668542,2.958200,1.483040
"support vector, raw",9.220157,4.773673,4.599977,4.489521,0.898413,1.256970,5.853871
"neural net (relu), raw",6.044525,2.644396,3.885643,2.935425,2.865852,3.714910,5.957046
"neural net (tanh), raw",7.293785,3.041862,3.045869,3.052894,2.806849,3.211177,7.046512
"neural net (log), raw",9.284244,5.452486,4.889343,4.718084,2.440279,2.471983,5.105930
"lasso, raw",5.487903,2.919502,2.906211,3.280754,2.997269,2.728938,69.477191
"gradient boosting, smooth",4.135849,2.095208,6.389912,4.097908,3.457031,1.684926,2.483468
"random forest, smooth",5.169065,2.045268,5.016283,4.173139,4.040695,1.371303,0.981301


,1,2,3,4,5,6,7
model for butyrate,,,,,,,
"gradient boosting, raw",4.375470,2.708151,3.484413,5.381249,4.031566,5.643303,9.849789
"random forest, raw",3.375282,2.771225,2.765102,4.599302,2.817933,6.246375,8.808901
"support vector, raw",4.372311,3.621919,2.578157,3.763310,5.673401,6.495065,10.908601
"neural net (relu), raw",5.190211,4.026007,4.974283,3.839948,4.280862,5.550572,12.654400
"neural net (tanh), raw",5.049905,4.462547,5.082990,3.829042,4.022185,4.741722,15.528787
"neural net (log), raw",4.739470,4.024811,2.607405,3.053821,7.032923,7.687513,11.316256
"lasso, raw",4.401314,5.554498,3.113632,5.568629,5.272392,6.774172,39.723962
"gradient boosting, smooth",4.146351,2.443458,2.196328,4.490697,1.829143,7.317413,8.907025
"random forest, smooth",4.282077,2.053534,2.330915,4.409994,1.671627,6.008288,8.108554


,1,2,3,4,5,6,7
model for ethanol,,,,,,,
"gradient boosting, raw",35.977346,9.874980,21.057444,30.640670,17.116824,42.678563,8.679744
"random forest, raw",31.526393,9.889421,21.920098,30.489338,19.502134,42.654227,5.792513
"support vector, raw",8.751151,20.965838,40.874206,29.167746,14.709441,15.093469,14.343261
"neural net (relu), raw",22.817362,13.892814,28.667830,25.549481,10.321444,17.132348,275.209300
"neural net (tanh), raw",8.003554,25.229586,44.021522,31.662854,11.097487,10.402275,14.479447
"neural net (log), raw",10.842944,29.923224,48.224442,36.073724,5.505592,5.632342,16.653932
"lasso, raw",42.945223,8.071713,22.303790,35.853110,25.522980,30.754885,139.060058
"gradient boosting, smooth",35.494514,14.097955,22.533987,33.182085,12.282929,42.170817,3.499219
"random forest, smooth",36.388003,5.392090,22.461580,35.496027,6.501587,42.719462,11.192876
